# Port Congestion API Example Queries.

## Find that example in [Colab](https://colab.research.google.com/drive/1E-PoWlPMz0OiJpZfdP1-T5Uf3mjYfzZi?authuser=1#scrollTo=xHmeLgGYDINd)

- Get your personal Signal Ocean API subscription key (acquired [here](https://apis.signalocean.com/profile)) and replace it below:

In [100]:
signal_ocean_api_key = ""  # replace with your subscription key

## Setup

- Install the Signal Ocean SDK and import all required modules:

In [101]:
%%capture
!pip install signal-ocean

from signal_ocean import Connection
connection = Connection(signal_ocean_api_key)

from signal_ocean import Connection
from signal_ocean.port_congestion.port_congestion_api import PortCongestionAPI

import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import date 
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

# Create instance of PortCongestionAPI used throughout this notebook:
connection = Connection(signal_ocean_api_key)
port_congestion_api = PortCongestionAPI(connection)

- Define Plot Port Congestion Helper Function

In [102]:
def congestion_plot(timeseries, title=""):
  fig = sp.make_subplots(
    rows=3, 
    cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.07,
    x_title="Observation Date",
    subplot_titles=("Vessels at Port", "Arrivals & Departures", "Estimated Average Wait")
  )

  # Queue
  fig.add_trace(
    go.Line(
      x=timeseries.index, 
      y=timeseries["queue"], 
      name="Vessels at Port", 
      line={"color": "orange", "width": 2}
    ), row=1, col=1
  )

  fig.update_yaxes(title_text="Vessels", row=1, col=1)
  
  # Arrivals/Departures
  fig.add_trace(
    go.Line(
      x=timeseries.index,
      y=timeseries["arrivals"],
      name="Arrivals",
      line={"color": "green", "width": 2}
    ),
    row=2, col=1
  )
  
  fig.append_trace(
    go.Line(
      x=timeseries.index,
      y=timeseries["departures"],
      name="Departures",
      line={"color": "blue", "width": 2}
    ), row=2, col=1
  )

  fig.update_yaxes(title_text="Vessels", row=2, col=1)

  # Wait Estimate
  fig.add_trace(
    go.Line(
      x=timeseries.index,
      y=timeseries["avg_wait_estimate"],
      name="Estimated Wait",
      line={"color": "red", "width": 2}
    ), row=3, col=1
  )

  fig.update_yaxes(title_text="Days", row=3, col=1)

  # Show Plot
  fig.update_layout(width=1000, height=500, title_text=title)
  fig.show()

## Query: Capes in "China/Taiwan" from Jan 1st, 2021 to Today

In [103]:
dt = date.fromisoformat("2021-01-01")
response = port_congestion_api.query_port_congestion(level_1_areas=["China / Taiwan"], vessel_classes=["Capesize"], date_from=dt)

ts_df = pd.DataFrame(response)
ts_df ["observation_date"] = ts_df["observation_date"].dt.date
ts_df .set_index("observation_date", inplace=True)

display(ts_df)

,queue,arrivals,departures,avg_days_since_arrival,avg_wait_estimate
observation_date,,,,,
2021-01-01,176,12,19,6.9,6.0
2021-01-02,170,13,20,6.9,6.3
2021-01-03,175,25,22,6.4,6.3
2021-01-04,174,21,25,6.5,6.3
2021-01-05,170,21,21,6.3,6.4
...,...,...,...,...,...
2023-05-05,97,18,21,2.9,2.5
2023-05-06,102,26,25,2.7,2.5
2023-05-07,105,28,18,2.2,2.4


In [104]:
congestion_plot(ts_df, title="Congestion: Capes in China / Taiwan")

## Query: Aframax in "Rotterdam" from Jan 1st, 2023t to Today

In [105]:
dt = date.fromisoformat("2021-01-01")
response = port_congestion_api.query_port_congestion(ports=["Rotterdam"], vessel_classes=["Aframax"], date_from=dt)

afrt_df = pd.DataFrame(response)
afrt_df["observation_date"] = afrt_df["observation_date"].dt.date
afrt_df.set_index("observation_date", inplace=True)

display(afrt_df)

,queue,arrivals,departures,avg_days_since_arrival,avg_wait_estimate
observation_date,,,,,
2021-01-01,10,4,2,2.4,2.6
2021-01-02,11,3,1,1.3,2.3
2021-01-03,13,3,3,1.6,2.1
2021-01-04,12,2,3,1.9,2.1
2021-01-05,14,5,1,1.8,2.0
...,...,...,...,...,...
2023-05-05,4,1,1,4.8,4.8
2023-05-06,8,5,1,2.4,4.2
2023-05-07,10,3,3,2.2,3.7


In [106]:
congestion_plot(afrt_df, title="Congestion: Afras in Rotterdam")